# Discrete areas

Split area into discrete regions.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns
from h3 import h3

from pylab import rcParams
rcParams['figure.figsize'] = (10, 7)

import pandas as pd
pd.set_option('max_columns', 40)
pd.set_option('max_rows', 20)
pd.set_option('precision', 3)

## Prepare data

In [2]:
%%time 

data = pd.read_feather('structured_data.feather')

data = data.sample(100000)

data.head()

CPU times: user 5.25 s, sys: 6.37 s, total: 11.6 s
Wall time: 16.8 s


,datetime,lon,lat
14860328,2015-02-12 12:00:54,-73.979,40.778
39222511,2015-04-18 16:08:31,-73.974,40.757
32431434,2015-03-31 10:00:08,-74.001,40.741
42284032,2015-04-26 15:08:15,-73.989,40.778
28248578,2015-03-19 23:29:54,-73.916,40.772


## Points to h3 indexes

In [5]:
def point_h3_indexes(points: pd.DataFrame, res: int) -> np.ndarray:
    h3_indexes = points.apply(
        lambda point: h3.geo_to_h3(point['lat'], point['lon'], res),
        axis=1
    )
    return h3_indexes
    
resolution = 8
h3_indexes_res_8 = point_h3_indexes(data, resolution)

resolution = 9
h3_indexes_res_9 = point_h3_indexes(data, resolution)

## Interactive map

Create an interactive map with unique hexagons

In [32]:
from shapely.geometry import Polygon
import geojson
from geojson import FeatureCollection
import folium

def indexes_to_polygons(h3_indexes):
    polygons = []
    
    for h in h3_indexes:
        polygon = Polygon(h3.h3_to_geo_boundary(h3_address=h, geo_json=True))
        polygons.append(polygon)
        
    feature_collection = FeatureCollection(polygons)
    return geojson.dumps(feature_collection)
        
def create_style_fn(color, opacity=0.1, weight=2):
    """Ugly way to set color of json features"""
    
    return lambda feature: {
        'fillColor': None,
        'color': color,
        'weight': weight,
        'fillOpacity': opacity
    }

m = folium.Map(location=[40.7536, -73.9811], zoom_start=12)

# resolution 8
feature_collection = indexes_to_polygons(h3_indexes_res_8.unique())
folium.GeoJson(
    feature_collection, 
    style_function=create_style_fn('blue', opacity=0.1, weight=2), 
    ).add_to(m)

# resolution 9
feature_collection = indexes_to_polygons(h3_indexes_res_9.unique())
folium.GeoJson(
    feature_collection, 
    style_function=create_style_fn('green', opacity=0., weight=1)
    ).add_to(m)

m

In [18]:
folium.GeoJson?